In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

In [3]:
df = pd.read_csv('diabetes.csv')
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

In [50]:
from scipy.stats import uniform, randint

param_dist = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'max_features': ["sqrt", "log2", None, 0.7, 0.75, 0.8],
    'subsample': uniform(0.6, 0.4),
    'reg_alpha': uniform(0, 1000),
    'reg_lambda': uniform(0, 1000)
}

xgb = XGBClassifier(
    objective='binary:logistic',
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

rs = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=500,
    scoring='roc_auc',
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

start_rs = time.time()
rs.fit(X_train, y_train)
time_rs = time.time() - start_rs

y_pred_rs = rs.best_estimator_.predict_proba(X_test)[:, 1]
auc_rs = roc_auc_score(y_test, y_pred_rs)

print("RandomizedSearchCV")
print(f"Лучшие параметры: {rs.best_params_}")
print(f"ROC-AUC (тест): {auc_rs:.4f}")
print(f"Время: {time_rs:.2f} сек")

RandomizedSearchCV
Лучшие параметры: {'learning_rate': 0.19962694885187268, 'max_depth': 7, 'max_features': 0.75, 'n_estimators': 339, 'reg_alpha': 6.08874823983907, 'reg_lambda': 100.94119997752149, 'subsample': 0.947577700911658}
ROC-AUC (тест): 0.8119
Время: 15.86 сек


D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_features", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [52]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope

In [54]:
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    model = XGBClassifier(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        objective='binary:logistic',
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)
    
    return {'loss': -auc, 'status': STATUS_OK}

In [56]:
space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 50, 500)),
    'max_depth': scope.int(hp.uniform('max_depth', 3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.31),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1000),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1000)
}

In [58]:
trials = Trials()
start_ho = time.time()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=500,
    trials=trials,
    rstate=np.random.default_rng(42)
)
time_ho = time.time() - start_ho

best_params_ho = {
    'n_estimators': int(best['n_estimators']),
    'max_depth': int(best['max_depth']),
    'learning_rate': best['learning_rate'],
    'subsample': best['subsample'],
    'reg_alpha': best['reg_alpha'],
    'reg_lambda': best['reg_lambda']
}

xgb_ho = XGBClassifier(
    **best_params_ho,
    objective='binary:logistic',
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_ho.fit(X_train, y_train)
y_pred_ho = xgb_ho.predict_proba(X_test)[:, 1]
auc_ho = roc_auc_score(y_test, y_pred_ho)

print("\nHyperopt (TPE)")
print(f"Лучшие параметры: {best_params_ho}")
print(f"ROC-AUC (тест): {auc_ho:.4f}")
print(f"Время: {time_ho:.2f} сек")

  1%|▎                                                             | 3/500 [00:00<00:16, 29.90trial/s, best loss: -0.5]

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARN

  2%|▊                                               | 9/500 [00:00<00:23, 20.87trial/s, best loss: -0.743044776119403]

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:15] WARN

  2%|█▏                                             | 12/500 [00:00<00:24, 19.59trial/s, best loss: -0.743044776119403]

...

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:45] WARN

100%|█████████████████████████████████████████████| 500/500 [00:30<00:00, 16.40trial/s, best loss: -0.8275820895522388]

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)

D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)





Hyperopt (TPE)
Лучшие параметры: {'n_estimators': 378, 'max_depth': 3, 'learning_rate': 0.2568833932159974, 'subsample': 0.6842638822835189, 'reg_alpha': 0.7842797224286262, 'reg_lambda': 544.8863154095927}
ROC-AUC (тест): 0.8276
Время: 30.49 сек


D:\Anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:19:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Было пройдлено 500 итераций. Hyperopt дал лучший результат по ROC_AUC с разницей в 1.5%, что важно в медицинских задачах, но он Hyperopt медленее RandomizedSearchCV в 2 раза.
Найденные гиперпараметры также отличаются. Hyperopt выбрал устойчивую к переобучению модель с небольшими деревьями и сильной регуляризацией, а RandomizedSearchCV наоборот.
Для задач с ограниченным числом объектов и при необходимости максимизации качества предпочтительнее использовать Hyperopt, даже несмотря на большее время оптимизации